In this notebook data from merged .csv (vibration and temperature) are loaded and processed by 'utils.data_utils' functions. In the end all the processed data is saved in binary format

In [1]:
import numpy as np

from utils import data_model
from utils.preprocessing import cumsum, hankel_svdvals, correlation_coeffs
from utils.preprocessing_c import derivative


%load_ext autoreload
%autoreload 2

In [2]:
dataset_path = 'data/processed_data/femto_dataset'

# 'all' or a list of bearings name (e.g. 'all' or ['Bearing1_1', 'Bearing2_5', 'Bearing3_1', ...])
bearings_to_load = 'all'

# 'all' or a list of data names (e.g. 'all' or ['acc', 'temp', 'cumsum', 'fft_spectogram', ...])
# results from 'data_utils' functions have the same data name of its corresponding function 
data_to_load = ['acc']

bearings = data_model.load(dataset_path, bearings_to_load, data_to_load)

---

Defining kwargs for preprocessing functions

In [16]:
fs = 25600                                 # 25600 Hz
recording_time = 0.1                       # 0.1 seconds
pts_per_recording = int(fs*recording_time) # 2560 points per recording


hankel_svdvals_kwargs = {'hankel_window_size' : 9,
                         'slice_window_size' : pts_per_recording}

correlation_coeffs_kwargs = {'baseline' : 0,
                             'norm_interval' : [-1, 1],
                             'filter_window' : 3,
                             'filter_polyorder' : 2}       

In [17]:
for bearing in bearings:
    
    # Vibration data
    acc_v = bearing.data['acc']['v']
    
    # Preprocessing functions 
    #bearing.data['cumsum_v'] = cumsum(acc_v)
    
    bearing.data['hankel_svdvals_v'] = hankel_svdvals(acc_v, **hankel_svdvals_kwargs)
        
    bearing.data['correlation_coeffs_v'] = correlation_coeffs(bearing.data['hankel_svdvals_v'],
                                                              **correlation_coeffs_kwargs)
    
    # Saving processed data
    bearing.save_data()

In [ ]:
import matplotlib.pyplot as plt

print("Hankel window size: %s" % hankel_svdvals_kwargs)
for bearing in bearings:
    
    print("Bearing %s" % bearing.name)
    print(min(bearing.data['correlation_coeffs_v']))
    plt.figure()
    plt.plot(bearing.data['correlation_coeffs_v'])